<a href="https://colab.research.google.com/github/KubraErensoy/BTK-DATASCIENCE/blob/main/ML_FeatureSelection_CrossValidation_code7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Variance-based feature selection
from sklearn.feature_selection import VarianceThreshold

x = [[0,0,1],[0,1,0],[1,0,0],[0,1,1],[0,1,0],[0,1,1]]
x

[[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]

In [2]:
sel = VarianceThreshold(threshold=0.8*(1-0.8)) # var(x)=p(1-p)
sel.fit_transform(x)

#ilk sutunun elenmesini bekliyoruz
#zira orada 0 değeri olma olasılığı 5/6 >0.8

array([[0, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1]])

In [3]:
#İstatiksel Model Seçimi
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X, y = load_iris(return_X_y=True)
X.shape

(150, 4)

In [4]:
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
X_new.shape

(150, 2)

In [5]:
#Model Temelli Öznitelik Seçimi
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
X, y = load_iris(return_X_y=True)
X.shape

(150, 4)

In [6]:
# L1 normu temelli öznitelik seçimi
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
X_new.shape

(150, 3)

In [7]:
# L2 normu temelli öznitelik seçimi
lsvc = LinearSVC(C=0.01, penalty="l2", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
X_new.shape

(150, 2)

In [8]:
#Ağaç Temelli Öznitelik Seçimi
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
X, y = load_iris(return_X_y=True)
X.shape

(150, 4)

In [9]:
clf = ExtraTreesClassifier(n_estimators=50)#50 farklı ağaç oluşturur
clf = clf.fit(X, y)
clf.feature_importances_  

array([0.09213755, 0.06066222, 0.41251365, 0.43468657])

In [10]:
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X_new.shape   

(150, 2)

**MODEL SELECTION**


In [11]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns

from sklearn.datasets import load_iris

In [12]:
iris = load_iris()

x = iris.data
y= iris.target

In [13]:
#normalizasyon
x =(x - np.min(x)) / (np.max(x) - np.min(x))

In [14]:
#eğitim test bölünmesi
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.30)

**1- Model Training with Cross Validation(Çapraz Doğrulama)**

  - Çapraz doğrulamada kullanacağımız makine öğrenmesi modelinde K-en komşu modelini seçiyoruz.


In [15]:
#3 En yakın komşu 
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) # k=n_neighbours
knn

KNeighborsClassifier(n_neighbors=3)

In [19]:
#10 fold çapraz doğrulama yapalım
from sklearn.model_selection import cross_val_score
fold_sayisi = 10
dogruluklar = cross_val_score(estimator=knn, X = x_train,y=y_train,cv=fold_sayisi)
print("Ortalama Doğruluk: ",np.mean(dogruluklar))
print("Doğrulukların Standart Sapması ",np.std(dogruluklar))

Ortalama Doğruluk:  0.9172727272727272
Doğrulukların Standart Sapması  0.0859030781926655


**Model Selection**

In [20]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns

from sklearn.datasets import load_iris

In [22]:
#veri yükle
iris = load_iris()

x = iris.data #özellikler
y= iris.target #sınıf

In [26]:
#normalizasyon
x =(x - np.min(x)) / (np.max(x) - np.min(x))


In [27]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.30)

**2-Grid Search Cross Validation (Izgara Araması Çapraz Doğrulama Uygulama)**



In [29]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) # k=n_neighbours

In [30]:
from sklearn.model_selection import GridSearchCV

grid = {"n_neighbors":np.arange(1,50)}
knn = KNeighborsClassifier()

knn_cv = GridSearchCV(knn,grid,cv=10) # GridSerachCV
knn_cv.fit(x,y)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])})

In [31]:
print("En iyi K Değeri: ",knn_cv.best_params_)
print("En iyi K Değerine Göre Doğruluk Değeri",knn_cv.best_score_)

En iyi K Değeri:  {'n_neighbors': 13}
En iyi K Değerine Göre Doğruluk Değeri 0.9800000000000001


In [33]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

grid = {"C":np.logspace(-3,3,7),"penalty":["l1","l2"]} #L1=Lasso ve L2=ridge

logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(x,y)


print("En iyi Hiper Parametre",logreg_cv.best_params_)
print("En iyi Hiper Parametreye Göre Doğruluk",logreg_cv.best_score_)

En iyi Hiper Parametre {'C': 1000.0, 'penalty': 'l2'}
En iyi Hiper Parametreye Göre Doğruluk 0.9800000000000001
